In [1]:
!pwd
!ls
!rm -r *
!ls
!git clone 'https://github.com/giangbang/traffic-flow-simulator.git'

In [ ]:
!add-apt-repository ppa:sumo/stable -y
!apt-get update -y
!apt-get install sumo sumo-tools sumo-doc
!pip install traci

In [ ]:
import os
os.environ['SUMO_HOME'] = "/usr/share/sumo/"

In [20]:
!python main.py --episode 1 --train --stop --nogui

Loading configuration ... done.
Step #0.00 (0ms ?*RT. ?UPS, TraCI: 3ms, vehicles TOT 0 ACT 0 BUF 0)                      

Loading configuration ... done.
Step #0.00 (0ms ?*RT. ?UPS, TraCI: 3ms, vehicles TOT 0 ACT 0 BUF 0)                       
Step #100.00 (0ms ?*RT. ?UPS, TraCI: 2ms, vehicles TOT 33 ACT 11 BUF 22)                   

added SUMO_HOME to tools directory
{'nogui': True, 'train': True, 'episode': 1, 'yellow_phase': 5, 'green_phase': 10, 'stop': True}
device available:  cpu
0 model loaded successfully
{'0': 8}
{'0': 3}
Episode 0
Done saving 0
total reward: [[0.]]
minimum reward: [[0.]]
maximum reward: [[0.]]
step:  100
